In [1]:
#meta:tag=hide
%env METAFLOW_PROFILE=dev-valay
%env METAFLOW_UI_URL=


env: METAFLOW_PROFILE=dev-valay
env: METAFLOW_UI_URL=


In [2]:
#meta:tag=hide
import os
os.makedirs("temp_files", exist_ok=True)


# `@model` Usage Patterns


<!-- START doctoc -->
<!-- END doctoc -->



## Key Features
- Model Saving: Save models with unique identifiers and metadata
- Object Loading: Load objects saved with `current.model.save()`/ `current.checkpoint.save()`/ `current.huggingface_hub.snapshot_download()`
- Version Control: Track different versions of models to derive lineage
- Metadata Management: Attach metadata about saved/loaded models    

## Saving Models

`@model` decorator injects a `model` object in `current.` Users can call `current.model.save()` to save models. This returns a dictionary (`ModelArtifact`) containing metadata about the saved model. This dictionary contains several key pieces of information that help track and identify the saved model.

The reference contains the following keys:
```python
{
    "key": str,          # Unique storage key/path for the model
    "uuid": str,         # Unique identifier for the model
    "type": str,         # Type of artifact (always "model")
    "pathspec": str,     # Metaflow pathspec (flow/run/step/task)
    "attempt": int,      # Attempt number of the task
    "created_on": str,   # ISO format timestamp of when model was saved
    "size": int,         # Size of the saved model in bytes
    "url": str,          # Full URL/path to the stored model
    "metadata": dict,    # User-provided metadata (optional)
    "label": str        # User-provided label (optional)
}
```

In [3]:
%%writefile temp_files/model_save_basic.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current
import os  #meta_hide_line
import json  #meta_hide_line

class ModelSavingFlow(FlowSpec):
    
    @model
    @step
    def start(self):
        import uuid
        # ... Create some model files ...
        with open("model.h5", "w") as f: #meta_hide_line
            f.write("model weights v1 "+ uuid.uuid4().hex) #meta_hide_line
            
        self.basic_model = current.model.save(
            "model.h5",
            label="basic_model",
            metadata={
                "version": "1.0",
                "accuracy": 0.95
            }
        )
        
        # ... save model into some directory ...
        os.makedirs("model_directory", exist_ok=True) #meta_hide_line
        with open("model_directory/weights.txt", "w") as f: #meta_hide_line
            f.write("model weights v2") #meta_hide_line
        with open("model_directory/config.json", "w") as f: #meta_hide_line
            json.dump({"layers": 3}, f) #meta_hide_line
            
        self.directory_model = current.model.save(
            "./model_directory",
            label="keras_model",
            metadata={
                "loss": "2.0",                
            }
        )
        
        # Store just the key for later use
        self.model_key = self.directory_model["key"]
        print(f"Model key: {self.model_key}")
        self.next(self.end)

    @step  #meta_hide_line
    def end(self):  #meta_hide_line
        pass  #meta_hide_line

if __name__ == "__main__":  #meta_hide_line
    ModelSavingFlow()  #meta_hide_line

Writing temp_files/model_save_basic.py


In [4]:
#meta:tag=hide_input
#meta:show_steps=start
! python temp_files/model_save_basic.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelSavingFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:09:32.243 Workflow starting (run-id 7454):


2024-12-11 06:09:33.227 [7454/start/47442 (pid 2203585)] Task is starting.


2024-12-11 06:09:36.692 [7454/start/47442 (pid 2203585)] Model key: mf.models/models/artifacts/keras_model_5680bf7d3344450697a1f340900f4878


2024-12-11 06:09:44.112 [7454/start/47442 (pid 2203585)] Task finished successfully.


2024-12-11 06:09:44.421 [7454/end/47443 (pid 2204272)] Task is starting.


2024-12-11 06:09:46.411 [7454/end/47443 (pid 2204272)] Task finished successfully.


2024-12-11 06:09:46.531 Done!


## Loading Models

The `@model` decorator can be used to load models from previous steps. The `load` argument can be given the name of the Metaflow DataArtifact that contains the return value of `current.model.save()`/ `current.checkpoint.save()` / `current.huggingface_hub.snapshot_download()`. The path to the loaded models will be accessible via `current.model.loaded`.

`current.model.loaded` is a dictionary-like object that manages loaded models. It provides access to loaded model paths and their associated metadata. The The `current.model.loaded.info` property provides access to model metadata of the loaded models.


In [5]:
%%writefile temp_files/model_load_basic.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current
import os  #meta_hide_line
import json  #meta_hide_line

class ModelSavingFlow(FlowSpec):
    
    @model
    @step
    def start(self):
        import uuid #meta_hide_line
        # ... Create some model files ...
        with open("model.h5", "w") as f: #meta_hide_line
            f.write("model weights v1 "+ uuid.uuid4().hex) #meta_hide_line
            
        self.basic_model = current.model.save(
            "model.h5",
            label="basic_model",
            metadata={
                "accuracy": 0.95
            }
        )
        
        self.next(self.end)

    @model(load=["basic_model"]) # Reference the model set to self
    @step  
    def end(self): 
        model_directory = current.model.loaded["basic_model"]
            # Access the path to the file like this:
        model_path = os.path.join(
            model_directory,
            "model.h5"
        )
        print("Model Path %s Exists: %s" % (model_path, os.path.exists(model_path)))
        print("Loaded model from pathspec: %s" % current.model.loaded.info["basic_model"]["pathspec"])


if __name__ == "__main__":  #meta_hide_line
    ModelSavingFlow()  #meta_hide_line

Writing temp_files/model_load_basic.py


In [6]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_basic.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelSavingFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:09:51.723 Workflow starting (run-id 7455):


2024-12-11 06:09:52.709 [7455/start/47445 (pid 2204584)] Task is starting.


...

2024-12-11 06:10:01.687 [7455/end/47447 (pid 2204922)] [@model] Loading Artifact with name `basic_model` [type:model] with key: mf.models/models/artifacts/basic_model_a60f0fbea2264e558114de5a169bc73f


2024-12-11 06:10:01.739 [7455/end/47447 (pid 2204922)] [@model] Loaded artifact `basic_model[type:model]` in 0.05 seconds
2024-12-11 06:10:01.764 [7455/end/47447 (pid 2204922)] Model Path /tmp/metaflow_models_basic_model_sr3vtg2v/model.h5 Exists: True


2024-12-11 06:10:05.609 [7455/end/47447 (pid 2204922)] Loaded model from pathspec: ModelSavingFlow/7455/start/47445


2024-12-11 06:10:05.773 [7455/end/47447 (pid 2204922)] Task finished successfully.


2024-12-11 06:10:05.886 Done!



### Loading HuggingFace Models

In [7]:
%%writefile temp_files/model_load_hf_basic.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current, huggingface_hub
import os

class HFModelFlow(FlowSpec):
    
    @huggingface_hub
    @step
    def start(self):
        self.hf_model = current.huggingface_hub.snapshot_download(
            repo_id="bert-base-uncased"
        )
        self.next(self.end)

    @model(load=["hf_model"]) # Reference the model set to self
    @step  
    def end(self): 
        model_path = current.model.loaded["hf_model"]
        print("Directory contents of hf_model: %s" % os.listdir(model_path))


if __name__ == "__main__":  #meta_hide_line
    HFModelFlow()  #meta_hide_line

Writing temp_files/model_load_hf_basic.py


In [8]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_hf_basic.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing HFModelFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:10:11.844 Workflow starting (run-id 7456):


2024-12-11 06:10:13.182 [7456/start/47450 (pid 2205302)] Task is starting.


...

2024-12-11 06:10:16.515 [7456/end/47451 (pid 2205399)] Task is starting.


2024-12-11 06:10:17.707 [7456/end/47451 (pid 2205399)] [@model] Loading Artifact with name `hf_model` [type:checkpoint] with key: mf.huggingface_hub/checkpoints/artifacts/HFModelFlow/start/26ec4b03ee0e/a92ae9615600/1a471abc.0.9b5c6e8800.0


2024-12-11 06:10:44.722 [7456/end/47451 (pid 2205399)] [@model] Loaded artifact `hf_model[type:checkpoint]` in 27.01 seconds
2024-12-11 06:10:44.746 [7456/end/47451 (pid 2205399)] Directory contents of hf_model: ['README.md', 'config.json', '.gitattributes', 'LICENSE', 'flax_model.msgpack', 'tf_model.h5', 'tokenizer.json', 'pytorch_model.bin', 'vocab.txt', 'model.onnx', 'rust_model.ot', 'model.safetensors', 'tokenizer_config.json']


2024-12-11 06:10:49.185 [7456/end/47451 (pid 2205399)] Task finished successfully.


2024-12-11 06:10:49.325 Done!


### Loading Checkpoints

In [9]:
%%writefile temp_files/model_load_chckpt_basic.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current, checkpoint
import os
import json

class CheckpointFlow(FlowSpec):
    
    @checkpoint
    @step
    def start(self):
        # ... Created some checkpoint in checkpoint_dir...
        os.makedirs("checkpoint_dir", exist_ok=True) #meta_hide_line
        with open("checkpoint_dir/weights.txt", "w") as f: #meta_hide_line
            f.write("model weights v2") #meta_hide_line
        with open("checkpoint_dir/config.json", "w") as f: #meta_hide_line
            json.dump({"layers": 3}, f) #meta_hide_line
            
        self.checkpoint_foo = current.checkpoint.save(
            "./checkpoint_dir",
            metadata={
                "version": "2.0",
            }
        )
        self.next(self.end)

    @model(load=["checkpoint_foo"]) # Reference the model set to self
    @step  
    def end(self): 
        model_path = current.model.loaded["checkpoint_foo"]
        print("Directory contents of hf_model: %s" % os.listdir(model_path))


if __name__ == "__main__":  #meta_hide_line
    CheckpointFlow()  #meta_hide_line

Writing temp_files/model_load_chckpt_basic.py


In [10]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_chckpt_basic.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing CheckpointFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:10:54.181 Workflow starting (run-id 7457):


2024-12-11 06:10:55.192 [7457/start/47456 (pid 2206111)] Task is starting.


...

2024-12-11 06:11:08.021 [7457/end/47458 (pid 2206295)] [@model] Loading Artifact with name `checkpoint_foo` [type:checkpoint] with key: mf.checkpoints/checkpoints/artifacts/CheckpointFlow/start/26ec4b03ee0e/a92ae9615600/54d4b3b1.0.mfchckpt.0


2024-12-11 06:11:08.985 [7457/end/47458 (pid 2206295)] [@model] Loaded artifact `checkpoint_foo[type:checkpoint]` in 0.96 seconds


2024-12-11 06:11:09.014 [7457/end/47458 (pid 2206295)] Directory contents of hf_model: ['config.json', 'weights.txt']


2024-12-11 06:11:13.160 [7457/end/47458 (pid 2206295)] Task finished successfully.


2024-12-11 06:11:13.380 Done!


### Loading to specific path

In [11]:
%%writefile temp_files/model_load_basic_specific_path.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current
import os  #meta_hide_line
import json  #meta_hide_line

class ModelSavingFlow(FlowSpec):
    
    @model
    @step
    def start(self):
        import uuid
        # ... Create some model files ...
        os.makedirs("model_directory", exist_ok=True) #meta_hide_line
        with open("model_directory/weights.txt", "w") as f: #meta_hide_line
            f.write("model weights v2") #meta_hide_line
        with open("model_directory/config.json", "w") as f: #meta_hide_line
            json.dump({"layers": 3}, f) #meta_hide_line
            
        self.basic_model = current.model.save(
            "./model_directory",
            label="basic_model",
            metadata={
                "version": "1.0",
                "accuracy": 0.95
            }
        )
        
        self.next(self.end)

    @model(load=[("basic_model", "./models/basic_model")]) # Reference the model set to self
    @step  
    def end(self): 
        model_directory = current.model.loaded["basic_model"]
        print("Model Path %s contains contents: %s" % (model_directory, os.listdir(model_directory)))
        print("Loaded model from pathspec: %s" % current.model.loaded.info["basic_model"]["pathspec"])


if __name__ == "__main__":  #meta_hide_line
    ModelSavingFlow()  #meta_hide_line

Writing temp_files/model_load_basic_specific_path.py


In [12]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_basic_specific_path.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelSavingFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:11:18.299 Workflow starting (run-id 7458):


2024-12-11 06:11:19.257 [7458/start/47460 (pid 2206429)] Task is starting.


...

2024-12-11 06:11:28.434 [7458/end/47461 (pid 2206552)] [@model] Loading Artifact with name `basic_model` [type:model] with key: mf.models/models/artifacts/basic_model_5d1f468cd35142a685de10b117644da4


2024-12-11 06:11:28.499 [7458/end/47461 (pid 2206552)] [@model] Loaded artifact `basic_model[type:model]` in 0.06 seconds
2024-12-11 06:11:28.523 [7458/end/47461 (pid 2206552)] Model Path ./models/basic_model contains contents: ['config.json', 'weights.txt']


2024-12-11 06:11:32.374 [7458/end/47461 (pid 2206552)] Loaded model from pathspec: ModelSavingFlow/7458/start/47460


2024-12-11 06:11:32.564 [7458/end/47461 (pid 2206552)] Task finished successfully.


2024-12-11 06:11:32.683 Done!


### Loading Models from Keys

In [13]:
%%writefile temp_files/model_load_basic_key.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current
import os  #meta_hide_line
import json  #meta_hide_line

class ModelSavingFlow(FlowSpec):
    
    @model
    @step
    def start(self):
        import uuid
        # ... Create some model files ...
        os.makedirs("model_directory", exist_ok=True) #meta_hide_line
        with open("model_directory/weights.txt", "w") as f: #meta_hide_line
            f.write("model weights v2") #meta_hide_line
        with open("model_directory/config.json", "w") as f: #meta_hide_line
            json.dump({"layers": 3}, f) #meta_hide_line
            
        self.basic_model = current.model.save(
            "./model_directory",
            label="basic_model",
            metadata={
                "version": "1.0",
                "accuracy": 0.95
            }
        )
        self.model_key = self.basic_model["key"]
        
        self.next(self.end)

    @model(load=["model_key"]) # Reference the model set to self
    @step  
    def end(self): 
        model_directory = current.model.loaded["model_key"]
        print("Model Path %s contains contents:: %s" % (model_directory, os.listdir(model_directory)))
        print("Loaded model from pathspec: %s" % current.model.loaded.info["model_key"]["pathspec"])


if __name__ == "__main__":  #meta_hide_line
    ModelSavingFlow()  #meta_hide_line

Writing temp_files/model_load_basic_key.py


In [14]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_basic_key.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelSavingFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:11:37.596 Workflow starting (run-id 7459):


2024-12-11 06:11:38.406 [7459/start/47463 (pid 2206677)] Task is starting.


...

2024-12-11 06:11:47.085 [7459/end/47464 (pid 2206798)] [@model] Loading Artifact with name `model_key` [type:model] with key: mf.models/models/artifacts/basic_model_e1417661935b42e88efc7c14d06a0305


2024-12-11 06:11:47.134 [7459/end/47464 (pid 2206798)] [@model] Loaded artifact `model_key[type:model]` in 0.05 seconds
2024-12-11 06:11:47.154 [7459/end/47464 (pid 2206798)] Model Path /tmp/metaflow_models_model_key_nhb3wmsb contains contents:: ['config.json', 'weights.txt']


2024-12-11 06:11:50.737 [7459/end/47464 (pid 2206798)] Loaded model from pathspec: ModelSavingFlow/7459/start/47463


2024-12-11 06:11:50.936 [7459/end/47464 (pid 2206798)] Task finished successfully.


2024-12-11 06:11:51.078 Done!


### Explicitly Loading Models

In [15]:
%%writefile temp_files/model_load_basic_explicit.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, current
import os  #meta_hide_line
import json  #meta_hide_line

class ModelSavingFlow(FlowSpec):
    
    @model
    @step
    def start(self):
        import uuid
        # ... Create some model files ...
        os.makedirs("model_directory", exist_ok=True) #meta_hide_line
        with open("model_directory/weights.txt", "w") as f: #meta_hide_line
            f.write("model weights v2") #meta_hide_line
        with open("model_directory/config.json", "w") as f: #meta_hide_line
            json.dump({"layers": 3}, f) #meta_hide_line
            
        self.basic_model = current.model.save(
            "./model_directory",
            label="basic_model",
            metadata={
                "version": "1.0",
                "accuracy": 0.95
            }
        )
        
        self.next(self.end)

    @model
    @step  
    def end(self): 
        model_directory = current.model.load(self.basic_model)
        print("Model Path %s contains contents: %s" % (model_directory, os.listdir(model_directory)))
        print("Loaded model from pathspec: %s" % self.basic_model["pathspec"])


if __name__ == "__main__":  #meta_hide_line
    ModelSavingFlow()  #meta_hide_line

Writing temp_files/model_load_basic_explicit.py


In [16]:
#meta:tag=hide_input
#meta:show_steps=end
! python temp_files/model_load_basic_explicit.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelSavingFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:11:55.874 Workflow starting (run-id 7460):


2024-12-11 06:11:56.717 [7460/start/47467 (pid 2206931)] Task is starting.


...

2024-12-11 06:12:04.643 [7460/end/47468 (pid 2207078)] Task is starting.


2024-12-11 06:12:06.095 [7460/end/47468 (pid 2207078)] [@model] Loaded artifact `4ad1a6[type:model]` in 0.05 seconds


2024-12-11 06:12:10.520 [7460/end/47468 (pid 2207078)] Model Path /tmp/metaflow_models_4ad1a6_8tpih9kw contains contents: ['config.json', 'weights.txt']
2024-12-11 06:12:10.520 [7460/end/47468 (pid 2207078)] Loaded model from pathspec: ModelSavingFlow/7460/start/47467


2024-12-11 06:12:10.684 [7460/end/47468 (pid 2207078)] Task finished successfully.


2024-12-11 06:12:10.802 Done!


### Loading Models Across Flows

#### Loading from Event Triggered Flows

```python
from metaflow import FlowSpec, step, trigger_on_finish, current, model
@trigger_on_finish(flow='ModelRefreshFlow')
class InferenceFlow(FlowSpec):

    @model
    @step
    def start(self):
        print("Triggering run", current.trigger.run)
        model_path = current.model.load(current.trigger.run.data.model_reference)
        # The ModelRefreshFlow contained a `self.model_artifact` set to the return value of 
        # `current.model.save()`/ `current.checkpoint.save()` / `current.huggingface_hub.snapshot_download()`. 
        print("Model path", model_path)
        self.next(self.end)

    @step
    def end(self):
        pass
```

#### Loading Models with Metaflow Client API

In [17]:
%%writefile temp_files/model_load_client_api.py
#meta:tag=hide_output
from metaflow import Flow, current, namespace, model, FlowSpec, step
from metaflow.client import Step, Task
import os
import time  #meta_hide_line

class InferenceFlow(FlowSpec):
    @step
    def start(self):
        # Get model key from previous flow
        run = Flow('ModelSavingFlow').latest_successful_run
        self.upstream_model = run.data.basic_model
        self.next(self.load_model)
    
    @model(load=["upstream_model"])
    @step
    def load_model(self):
        # Decorator-based loading
        model_path = current.model.loaded["upstream_model"]
        print(f"Loaded via decorator to: {model_path} with contents: {os.listdir(model_path)}")
        
        time.sleep(5) #meta_hide_line
        # Manual loading
        manual_path = current.model.load(
            self.upstream_model,
            path="./manual_load"
        )
        print(f"Loaded manually to: {manual_path} with contents: {os.listdir(manual_path)}")
        self.next(self.end)
        
    @step
    def end(self):
        pass

if __name__ == "__main__":  #meta_hide_line
    InferenceFlow()  #meta_hide_line


Writing temp_files/model_load_client_api.py


In [18]:
#meta:tag=hide_input
#meta:show_steps=load_model
! python temp_files/model_load_client_api.py run 

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing InferenceFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:12:15.437 Workflow starting (run-id 7461):


2024-12-11 06:12:16.402 [7461/start/47470 (pid 2207204)] Task is starting.


...

2024-12-11 06:12:25.662 [7461/load_model/47471 (pid 2207244)] [@model] Loaded artifact `4ad1a6[type:model]` in 0.04 seconds
2024-12-11 06:12:25.662 [7461/load_model/47471 (pid 2207244)] Loaded manually to: ./manual_load with contents: ['config.json', 'weights.txt']


2024-12-11 06:12:30.405 [7461/load_model/47471 (pid 2207244)] Task finished successfully.


2024-12-11 06:12:30.841 [7461/end/47473 (pid 2207356)] Task is starting.


2024-12-11 06:12:32.725 [7461/end/47473 (pid 2207356)] Task finished successfully.


2024-12-11 06:12:32.834 Done!


#### Loading Using Parameters

Every model artifact saved with `current.model.save()` is assigned a unique key. This key is present in the `ModelArtifact` dictionary returned by `current.model.save()`. This can also be used to load models across flows.


In [19]:
%%writefile temp_files/model_load_key_param.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, model, Parameter, current

class ModelConsumerFlow(FlowSpec):
    # Model key is the key of the model artifact saved with current.model.save()
    # This key can be accessed via the `key` property of dictionary returned by 
    # `current.model.save()` / `current.checkpoint.save()` / `current.huggingface_hub.snapshot_download()`
    model_key = Parameter('model-key', 
                         help="Key of the model to load")

    @model(load=[("model_key", "./models/consumer")])
    @step
    def start(self):
        # Model is loaded to ./model directory
        import os
        print("Model files:", os.listdir("./models/consumer"))
        print("Loaded model from pathspec: %s" % current.model.loaded.info["model_key"]["pathspec"])
        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == "__main__":  #meta_hide_line
    ModelConsumerFlow()  #meta_hide_line

Writing temp_files/model_load_key_param.py


In [20]:
#meta:show_steps=start
! python temp_files/model_load_key_param.py run --model-key mf.models/models/artifacts/basic_model_828f6108847e44cf8e79d95781434788

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing ModelConsumerFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 06:12:37.563 Workflow starting (run-id 7462):


2024-12-11 06:12:38.472 [7462/start/47475 (pid 2207435)] Task is starting.


...

2024-12-11 06:12:39.823 [7462/start/47475 (pid 2207435)] Model files: ['config.json', 'weights.txt']
2024-12-11 06:12:39.823 [7462/start/47475 (pid 2207435)] Loaded model from pathspec: ModelSavingFlow/7083/start/45856


2024-12-11 06:12:43.714 [7462/start/47475 (pid 2207435)] Task finished successfully.


2024-12-11 06:12:44.074 [7462/end/47477 (pid 2207517)] Task is starting.


2024-12-11 06:12:45.945 [7462/end/47477 (pid 2207517)] Task finished successfully.


2024-12-11 06:12:46.060 Done!


### Loading Models After Flow Execution

In [21]:
from metaflow import Flow 
from metaflow import load_model
run = Flow('ModelSavingFlow').latest_successful_run
upstream_model = run.data.basic_model
load_model(upstream_model, path="./models/notebook-load")
print(
    "Model created for pathspec", run.data.basic_model["pathspec"]
)
print("Model files:", os.listdir("./models/notebook-load"))

Model created for pathspec ModelSavingFlow/7460/start/47467
Model files: ['config.json', 'weights.txt']
